In [39]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import re
import time
from tqdm import tqdm

In [40]:
# def best_weights(feature_df, results_df):
#     num_features = len(feature_df.columns[2:-1])
#     num_iterations = 2000
#     weights=[]
#     accuracy_list = []
#     weight_list = []
#     weights_df = pd.DataFrame()
#     results_range=results_df[results_df['Year'].isin(feature_df['Year'].unique())]
#     # performance_dict={}
#     # print(results_range)
#     i=0
#     for single_weight in tqdm(range(num_iterations)):
#         results_index=[]
#         temp_df=pd.DataFrame()
#         comparison_column=[]
#         weights = np.random.randint(low=-1, high=1, size=num_features).astype(np.float32)
#         # weights = np.random.random(num_features)
#         # weights /= np.sum(weights)
#         fran_weights = feature_df.iloc[:, 2:-3].T.apply(lambda x: x * weights[:-2])
#         fran_weights = fran_weights.T
#         fran_weights = fran_weights.multiply(feature_df['Strength of Schedule']*weights[-2], axis=0).multiply(feature_df['Last 10 Rating']*weights[-1], axis=0)
#         # print(fran_weights)
#         # fran_weights = fran_weights.drop(['Strength of Schedule', 'Last 10 Rating'], axis=1)
#         fran_weights['Scaled Sum'] = fran_weights.sum(axis=1)
#         fran_weights.insert(0,"Team",feature_df['Team'].values)
#         fran_weights.insert(len(fran_weights.columns), "Year", feature_df['Year'].values)
    
#         for index, row in results_range.iterrows():
#             # print(row['Year'])
#             # print(index, row)
#             team_1_weight = fran_weights[(fran_weights['Team']==row['Team 1']) & (fran_weights['Year']==row['Year'])]['Scaled Sum'].values
#             # print(team_1_weight)
#             team_2_weight = fran_weights[(fran_weights['Team']==row['Team 2']) & (fran_weights['Year']==row['Year'])]['Scaled Sum'].values
#             # print(team_2_weight)
#             if team_1_weight > team_2_weight:
#                 results_index.append(row['Team 1'])
#             else:
#                 results_index.append(row['Team 2'])
#         temp_df['Winner'] = results_range['Winner']
#         temp_df['Predicted Winner'] = results_index
#         # print(temp_df)
#         comparison_column = np.where(temp_df["Winner"] == temp_df["Predicted Winner"], 1, 0)
#         # print(sum(comparison_column))
#         accuracy = (sum(comparison_column)/temp_df.shape[0])*100
#         # print(performance_dict)
#         # print(len(performance_dict))
#         # print(accuracy, )
#         accuracy_list.append(accuracy)
#         weight_list.append(weights)
#         i+=1
#         pass
#     weights_df['Accuracy'] = accuracy_list
#     weights_df['Weights'] = weight_list
        
    
        
#     return(weights_df)


    


In [41]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

def best_weights(feature_df, results_df, num_iterations=5000, n_jobs=4):
    num_features = len(feature_df.columns[2:-1])
    results_range = results_df[results_df['Year'].isin(feature_df['Year'].unique())]
    
    def single_iteration(_):
        weights = np.random.uniform(-1, 1, size=num_features).astype(np.float32)
        fran_weights = feature_df.iloc[:, 2:-3].mul(weights[:-2], axis=1)
        fran_weights = fran_weights.mul(feature_df['Strength of Schedule'] * weights[-2], axis=0)
        fran_weights = fran_weights.mul(feature_df['Last 10 Rating'] * weights[-1], axis=0)
        fran_weights['Scaled Sum'] = fran_weights.sum(axis=1)
        fran_weights['Team'] = feature_df['Team'].values
        fran_weights['Year'] = feature_df['Year'].values
        
        results_index = []
        for _, row in results_range.iterrows():
            team_1_weight = fran_weights[(fran_weights['Team'] == row['Team 1']) & 
                                         (fran_weights['Year'] == row['Year'])]['Scaled Sum'].values
            team_2_weight = fran_weights[(fran_weights['Team'] == row['Team 2']) & 
                                         (fran_weights['Year'] == row['Year'])]['Scaled Sum'].values
            results_index.append(row['Team 1'] if team_1_weight > team_2_weight else row['Team 2'])
        
        comparison_column = np.where(results_range['Winner'] == results_index, 1, 0)
        accuracy = (np.sum(comparison_column) / len(comparison_column)) * 100
        return accuracy, weights
    
    results = Parallel(n_jobs=n_jobs)(delayed(single_iteration)(i) for i in tqdm(range(num_iterations)))
    accuracy_list, weight_list = zip(*results)
    
    weights_df = pd.DataFrame({'Accuracy': accuracy_list, 'Weights': weight_list})
    return weights_df

# Example usage:
# feature_df = pd.read_csv('feature_data.csv')
# results_df = pd.read_csv('results_data.csv')
# optimized_weights_df = best_weights(feature_df, results_df)

In [42]:
features_df = pd.read_csv("../data/2007-2025_MBB_Historical-Data.csv")
results_df = pd.read_csv("../data/Bracket_Historical_Data.csv")
# print(results_df, features_df)
weights_df = best_weights(features_df, results_df)
print(weights_df)

       Accuracy                                            Weights
0     34.369449  [0.5914612, 0.5761139, 0.46812233, -0.97352034...
1     68.117229  [-0.1522491, 0.21936657, -0.7976995, -0.662759...
2     67.140320  [0.37194937, 0.49837467, 0.967717, 0.3507743, ...
3     33.836590  [-0.45524514, 0.008178537, 0.44905505, 0.20340...
4     41.474245  [-0.9102562, -0.2177984, 0.61142045, 0.1856671...
...         ...                                                ...
4995  63.321492  [0.7187253, -0.5546268, -0.87305325, -0.032457...
4996  65.097691  [0.6847449, -0.62782925, -0.66972774, 0.907273...
4997  69.982238  [-0.15674675, 0.97846514, -0.5974673, -0.80231...
4998  36.944938  [-0.4529713, 0.38808918, 0.1425017, -0.3685091...
4999  53.463588  [-0.3163015, 0.22779045, -0.26838872, -0.99581...

[5000 rows x 2 columns]


In [43]:
# features_df = pd.read_csv("../data/2007-2025_MBB_Historical-Data.csv")
# results_df = pd.read_csv("../data/Bracket_Historical_Data.csv")
# # print(results_df, features_df)
# weights_df = best_weights(features_df, results_df)
# print(weights_df)

In [44]:
weights_df = weights_df.sort_values(by=['Accuracy'], ascending=False)

In [45]:
print(weights_df)

       Accuracy                                            Weights
585   71.758437  [0.7223505, -0.98704255, -0.084031686, -0.2064...
307   71.492007  [0.8491769, -0.17498298, 0.3896945, 0.64510274...
332   71.314387  [-0.7515824, -0.49981835, -0.0062137973, -0.67...
1361  71.225577  [-0.3122878, -0.16840461, -0.45278543, 0.81312...
3817  71.225577  [0.41853854, -0.43752155, 0.3457396, -0.131560...
...         ...                                                ...
4424  28.863233  [0.19516423, 0.020700645, -0.3948779, -0.86300...
3034  28.774423  [0.6086952, -0.46539044, -0.7362764, -0.161301...
4665  28.685613  [0.5306722, -0.032521125, 0.10292307, 0.162138...
4894  28.152753  [-0.6686844, 0.8656679, -0.10030574, 0.8643493...
158   28.063943  [-0.56566805, 0.8331059, -0.98621875, -0.97466...

[5000 rows x 2 columns]


In [47]:
weights = weights_df['Weights'][0]
results_index = 0
# print(features_df)
df = features_df[features_df['Year']==2025]
# print(df)
fran_weights = df.iloc[:, 2:-3].T.apply(lambda x: x * weights[:-2])
fran_weights=fran_weights.T
# fran_weights = fran_weights.multiply(df['Strength of Schedule']*weights[-2], axis=0).multiply(df['Last 10 Rating']*weights[-1], axis=0)
fran_weights['Scaled Sum'] = fran_weights.sum(axis=1)
fran_weights['Team'] = features_df['Team']
fran_weights['Year'] = features_df['Year']
fran_weights = fran_weights.sort_values(by=['Scaled Sum'], ascending=False)
fran_weights.to_csv('Prediction_sum.csv')
